In [184]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
import torch
import torchvision
import pytorch_lightning as pl

import os
import PIL.Image as Image
import base64
import io
import json
import requests
from urllib.parse import urlencode
from k12libs.utils.nb_easy import k12ai_get_top_dir, RACEURL

np.__version__, torch.__version__, torchvision.__version__, pl.__version__

('1.18.5', '1.6.0.dev20200609+cu101', '0.7.0.dev20200609+cu101', '1.0.8')

## Dataset 

### Garbage

In [220]:
model = 'Resnet18'
root_dir = '/raceai/data/tmp/pl_rgarbage_resnet18'
data_root = '/raceai/data/datasets/rgarbage/'
input_size = 224
mean = [
    0.6535,
    0.6132,
    0.5643
]
std = [
    0.2165,
    0.2244,
    0.2416
]
num_classes = 4

### Mnist

In [190]:
model = 'Resnet18'
root_dir = '/raceai/data/tmp/pl_rmnist_resnet18'
data_root = '/raceai/data/datasets/rmnist/'
input_size = 28
mean = [
    0.1362,
    0.1362,
    0.1362
]
std = [
    0.2893,
    0.2893,
    0.2893
]
num_classes = 10

## RestAPI

### Training

In [221]:
optimizer = {
    "class_name": "torch.optim.Adam",
    "params": {
        "lr": 0.001,
        "betas": [0.9, 0.999],
        "weight_decay": 0,
        "amsgrad": True
    }
}

# optimizer = {
#     "class_name": "torch.optim.SGD",
#     "params": {
#         "lr": 0.001,
#         "momentum": 0.9,
#         "weight_decay": 0,
#         "nesterov": True
#     }
# }
# 
# scheduler = {
#     "class_name": "torch.optim.lr_scheduler.StepLR",
#     "params": {
#         "step_size": 20,
#         "gamma": 0.1
#     }
# }

# scheduler = {
#     "class_name": "torch.optim.lr_scheduler.MultiStepLR",
#     "params": {
#         "milestones": [20, 40],
#         "gamma": 0.1
#     }
# }

scheduler = {
    "class_name": "torch.optim.lr_scheduler.ReduceLROnPlateau",
    "params": {
        "factor": 0.1,
        "patience": 3,
        "min_lr": 1e-6,
        "threshold": 1e-4,
        "verbose": True,
    }
}

In [227]:
early_stopping = {
    "class_name": "pytorch_lightning.callbacks.EarlyStopping",
    "params": {
        "monitor": "val_loss",
        "mode": "min",
        "patience": 6,
        "verbose": True,
        "min_delta": 0.0001
    }
}

model_checkpoint = {
    "class_name": "pytorch_lightning.callbacks.ModelCheckpoint",
    "params": {
        "monitor": "val_acc",
        "mode": "max",
        "save_top_k": 1,
        "period": 2,
        "save_weights_only": False,
        "filename": "best"
    }
}

logger = {
    "class_name": "pytorch_lightning.loggers.CSVLogger",
    "params": {
        "save_dir": root_dir,
        "name": "logs"
    }
}

random_rotation = {
    "class_name": "torchvision.transforms.RandomRotation",
    "params": {
        "degrees": 75
    }
}

reqdata = {
    "task": "cls.training.pl",
    "cfg": {
        "versions": {
            "torch": "1.7.0.dev20200819+cu101",
            "torchvision": "0.8.0.dev20200828+cu101",
            "pytorch_lightning": "1.0.4"
        },
        "data": {
            "train": {
                "class_name": "raceai.data.process.LocalDataLoader",  
                "params": {
                    "data_source": f"{data_root}/train.json",
                    "dataset": {
                        "class_name": "raceai.data.JsonFileDataset",
                         "params": {
                             "input_size": input_size,
                             "mean": mean,
                             "std": std,
                             "imgaugs": []
                         }
                    },
                    "sample": {
                        "batch_size": 32,
                        "num_workers": 4,
                        "drop_last": True,
                        "shuffle": True
                    }
                }
            },
            "valid": {
                "class_name": "raceai.data.process.LocalDataLoader",  
                "params": {
                    "data_source": f"{data_root}/val.json",
                    "dataset": {
                        "class_name": "raceai.data.JsonFileDataset",
                         "params": {
                             "input_size": input_size,
                             "mean": mean,
                             "std": std,
                         }
                     },
                     "sample": {
                        "batch_size": 32,
                        "num_workers": 4,
                        "drop_last": False
                    }
                 }
            },
            "test": {
                "class_name": "raceai.data.process.LocalDataLoader",  
                "params": {
                    "data_source": f"{data_root}/test.json",
                    "dataset": {
                        "class_name": "raceai.data.JsonFileDataset",
                         "params": {
                             "input_size": input_size,
                             "mean": mean,
                             "std": std,
                         }
                     },
                    "sample": {
                        "batch_size": 32,
                        "num_workers": 4,
                    }
                 }
            }
        },
        "model": {
            "class_name": f"raceai.models.backbone.{model}",  
            "params": {
                "device": 'gpu',
                "num_classes": num_classes,
                "weights": True,
            }
        },
        "solver": {
            "optimizer": optimizer,
            "scheduler": scheduler,
            "trainer": {
                "class_name": "raceai.runner.pl.PlTrainer",
                "params": {
                    "callbacks": [model_checkpoint],
                    "logger": False,
                    "general": {
                        "max_epochs": 8,
                        "gpus": [0],
                        "resume_from_checkpoint": None,
                        "default_root_dir": root_dir
                    }
                }
            }
        }
    }
}

uri = f'{RACEURL}/raceai/framework/training'

resdata = json.loads(requests.post(url=uri, json=reqdata).text)
if 0 != resdata['errno']:
    if 'traceback' in resdata['result']:
        print(resdata['result']['traceback'])
    else:
        print(resdata)
else:
    print(resdata)

Traceback (most recent call last):
  File "/raceai/codes/app/raceai/utils/misc.py", line 83, in _target
    queue.put(func(*args))
  File "/raceai/codes/app/raceai/runner/training.py", line 59, in pl_classifier_fit
    result = classifier.test(test_loader)
  File "/raceai/codes/app/raceai/runner/pl.py", line 102, in test
    return self.trainer.test(self, test_loader)
  File "/raceai/codes/projects/pytorch-lightning/pytorch_lightning/trainer/trainer.py", line 733, in test
    results = self.__test_given_model(model, test_dataloaders)
  File "/raceai/codes/projects/pytorch-lightning/pytorch_lightning/trainer/trainer.py", line 798, in __test_given_model
    results = self.fit(model)
  File "/raceai/codes/projects/pytorch-lightning/pytorch_lightning/trainer/trainer.py", line 445, in fit
    results = self.accelerator_backend.train()
  File "/raceai/codes/projects/pytorch-lightning/pytorch_lightning/accelerators/gpu_accelerator.py", line 63, in train
    results = self.train_or_test()
  Fi

### Inference

In [195]:
# resume_weights = f"/raceai/data/ckpts/rgarbage/pl_resnet18_acc85.pth"
# resume_weights = f"/raceai/data/ckpts/rmnist/pl_resnet18_rotation.pth"
resume_weights = f"/raceai/data/ckpts/rmnist/pl_resnet18_normal.pth"

reqdata = {
    "task": "cls.inference.pl",
    "cfg": {
        "data": {
            "class_name": "raceai.data.process.LocalDataLoader",  
            "params": {
                "data_source": f"{data_root}/test_rotation.json",
                "dataset": {
                    "class_name": "raceai.data.JsonFileDataset",
                     "params": {
                         "input_size": input_size,
                         "mean": mean,
                         "std": std
                     }
                 },
                "sample": {
                    "batch_size": 32,
                    "num_workers": 4,
                }
             }
        },
        "model": {
            "class_name": f"raceai.models.backbone.{model}",  
            "params": {
                "device": 'gpu',
                "num_classes": num_classes,
                "weights": False
            }
        },
        "trainer": {
            "default_root_dir": root_dir,
            "gpus": 1,
            "resume_from_checkpoint": resume_weights
        }
    }
}

uri = f'{RACEURL}/raceai/framework/inference'

resdata = json.loads(requests.post(url=uri, json=reqdata).text)
if 0 != resdata['errno']:
    if 'traceback' in resdata['result']:
        print(resdata['result']['traceback'])
    else:
        print(resdata)
else:
    print(resdata)

{'errno': 0, 'result': [{'test_acc': 0.6418651342391968}]}
